In [2]:
import os
from pathlib import Path
import pandas as pd
import json
import codecs
import numpy as np
import re
from collections import OrderedDict
from collections import Counter
import json

In [7]:
conll_train = '/home/pengfei/data/2015-2016_conll_shared_task/data/conll16st-en-03-29-16-train/pdtb-parses.json'
parse_dict_train = json.loads(codecs.open(conll_train, encoding='utf-8', errors='ignore').read())
conll_dev = '/home/pengfei/data/2015-2016_conll_shared_task/data/conll16st-en-03-29-16-dev/pdtb-parses.json'
parse_dict_dev = json.loads(codecs.open(conll_dev, encoding='utf-8', errors='ignore').read())
conll_test = '/home/pengfei/data/2015-2016_conll_shared_task/data/conll16st-en-03-29-16-test/pdtb-parses.json'
parse_dict_test = json.loads(codecs.open(conll_test, encoding='utf-8', errors='ignore').read())
print("datasets loaded")
parse_dict = merge3dicts(parse_dict_train, parse_dict_dev, parse_dict_test)

datasets loaded


## Sense

In [3]:
parse_data_path = '../pdtb3.json'
parse_data = [json.loads(o) for o in open(parse_data_path).readlines()]

In [3]:
counter = Counter([o for relation in parse_data for o in relation['Sense']])

In [4]:
sense = list(counter.most_common(100))

In [5]:
{o[0]:str(i) for i,o in enumerate(sense)}

{'Expansion.Conjunction': '0',
 'Comparison.Concession.Arg2-as-denier': '1',
 'EntRel': '2',
 'Contingency.Cause.Reason': '3',
 'Contingency.Cause.Result': '4',
 'Expansion.Level-of-detail.Arg2-as-detail': '5',
 'Temporal.Synchronous': '6',
 'Comparison.Contrast': '7',
 'Temporal.Asynchronous.Precedence': '8',
 'Expansion.Instantiation.Arg2-as-instance': '9',
 'Contingency.Condition.Arg2-as-cond': '10',
 'Contingency.Purpose.Arg2-as-goal': '11',
 'Temporal.Asynchronous.Succession': '12',
 'Comparison.Concession.Arg1-as-denier': '13',
 'Expansion.Manner.Arg2-as-manner': '14',
 'Expansion.Manner.Arg1-as-manner': '15',
 'Expansion.Substitution.Arg2-as-subst': '16',
 'Expansion.Equivalence': '17',
 'Expansion.Disjunction': '18',
 'Expansion.Level-of-detail.Arg1-as-detail': '19',
 'NoRel': '20',
 'Comparison.Similarity': '21',
 'Contingency.Cause+Belief.Reason+Belief': '22',
 'Hypophora': '23',
 'Contingency.Purpose.Arg1-as-goal': '24',
 'Expansion.Substitution.Arg1-as-subst': '25',
 'Conti

In [6]:
{str(i):o[0] for i,o in enumerate(sense)}

{'0': 'Expansion.Conjunction',
 '1': 'Comparison.Concession.Arg2-as-denier',
 '2': 'EntRel',
 '3': 'Contingency.Cause.Reason',
 '4': 'Contingency.Cause.Result',
 '5': 'Expansion.Level-of-detail.Arg2-as-detail',
 '6': 'Temporal.Synchronous',
 '7': 'Comparison.Contrast',
 '8': 'Temporal.Asynchronous.Precedence',
 '9': 'Expansion.Instantiation.Arg2-as-instance',
 '10': 'Contingency.Condition.Arg2-as-cond',
 '11': 'Contingency.Purpose.Arg2-as-goal',
 '12': 'Temporal.Asynchronous.Succession',
 '13': 'Comparison.Concession.Arg1-as-denier',
 '14': 'Expansion.Manner.Arg2-as-manner',
 '15': 'Expansion.Manner.Arg1-as-manner',
 '16': 'Expansion.Substitution.Arg2-as-subst',
 '17': 'Expansion.Equivalence',
 '18': 'Expansion.Disjunction',
 '19': 'Expansion.Level-of-detail.Arg1-as-detail',
 '20': 'NoRel',
 '21': 'Comparison.Similarity',
 '22': 'Contingency.Cause+Belief.Reason+Belief',
 '23': 'Hypophora',
 '24': 'Contingency.Purpose.Arg1-as-goal',
 '25': 'Expansion.Substitution.Arg1-as-subst',
 '26': 

In [7]:
[o[0] for o in sense]

['Expansion.Conjunction',
 'Comparison.Concession.Arg2-as-denier',
 'EntRel',
 'Contingency.Cause.Reason',
 'Contingency.Cause.Result',
 'Expansion.Level-of-detail.Arg2-as-detail',
 'Temporal.Synchronous',
 'Comparison.Contrast',
 'Temporal.Asynchronous.Precedence',
 'Expansion.Instantiation.Arg2-as-instance',
 'Contingency.Condition.Arg2-as-cond',
 'Contingency.Purpose.Arg2-as-goal',
 'Temporal.Asynchronous.Succession',
 'Comparison.Concession.Arg1-as-denier',
 'Expansion.Manner.Arg2-as-manner',
 'Expansion.Manner.Arg1-as-manner',
 'Expansion.Substitution.Arg2-as-subst',
 'Expansion.Equivalence',
 'Expansion.Disjunction',
 'Expansion.Level-of-detail.Arg1-as-detail',
 'NoRel',
 'Comparison.Similarity',
 'Contingency.Cause+Belief.Reason+Belief',
 'Hypophora',
 'Contingency.Purpose.Arg1-as-goal',
 'Expansion.Substitution.Arg1-as-subst',
 'Contingency.Negative-condition.Arg2-as-negCond',
 'Contingency.Condition+SpeechAct',
 'Contingency.Cause+Belief.Result+Belief',
 'Contingency.Condition

## write explicit conn

In [6]:
conn = Counter([relation['Connective']['RawText'] for relation in parse_data if relation['Type']=='Explicit' ])

In [7]:
len(conn), len([relation['Connective']['RawText'] for relation in parse_data if relation['Type']=='Explicit' ])

(170, 21548)

In [18]:
connective_list = [o[0] for o in conn.most_common()]

In [15]:
with open('/home/pengfei/Github/conll3_discourse/data/connective-category.txt') as f:
    conns_old = [o.split('#')[0].strip() for o in f.readlines()]

In [22]:
x = set(connective_list) - set(conns_old)
x

{'about',
 'afterwards',
 'albeit',
 'along with',
 'and then',
 'and/or',
 'as much as',
 'as well as',
 'at the same time',
 'because of',
 'beyond',
 'both and',
 'but also',
 'but then',
 'but then again',
 'by',
 'depending on',
 'despite',
 'due to',
 'even after',
 'even as',
 'even before',
 'even before then',
 'even if',
 'even so',
 'even then',
 'even though',
 'even when',
 'even while',
 'even with',
 'from',
 'given',
 'if only',
 'in',
 'in any case',
 'in case',
 'in order',
 'in the meantime',
 'in the meanwhile',
 'instead of',
 'later on',
 'like',
 'more accurately',
 'much less',
 'no matter',
 'not just but',
 'not just but also',
 'not only',
 'not only also',
 'not only because of',
 'not only but',
 'not only but also',
 'not so much as',
 'on',
 'on the one hand on the other',
 'only',
 'only if',
 'or otherwise',
 'rather than',
 'regardless of',
 'since before',
 'so as',
 'so long as',
 'so much as',
 'subsequently',
 'such as',
 'that is',
 'upon',
 'what

In [15]:
def read_file(filename):
    with open(filename) as f:
        rawtext = f.readlines()
    return [o.strip() for o in rawtext]
conn3_path = '/home/pengfei/Github/conll3_discourse/data/ExpConn.txt'
conn2_path = '/home/pengfei/Github/conll3_discourse/data/sortedExpConn.txt'
conn3 = read_file(conn3_path)
conn2 = read_file(conn2_path)
long = []
short = []
for conn in conn3:
    if '..' in conn:
        long.append(conn)
    else:
        short.append(conn)
overall = long + short
with open('sortedExpConn.txt', 'a') as f:
    for conn in overall:
        f.write(conn)
        f.write('\n')

# multi sense classification

In [4]:
def merge3dicts(x, y, z):
    m = x
    m.update(y)
    m.update(z)
    return m
# pdtb2
conll_train = '/home/pengfei/data/2015-2016_conll_shared_task/data/conll16st-en-03-29-16-train/pdtb-parses.json'
parse_dict_train = json.loads(codecs.open(conll_train, encoding='utf-8', errors='ignore').read())
conll_dev = '/home/pengfei/data/2015-2016_conll_shared_task/data/conll16st-en-03-29-16-dev/pdtb-parses.json'
parse_dict_dev = json.loads(codecs.open(conll_dev, encoding='utf-8', errors='ignore').read())
conll_test = '/home/pengfei/data/2015-2016_conll_shared_task/data/conll16st-en-03-29-16-test/pdtb-parses.json'
parse_dict_test = json.loads(codecs.open(conll_test, encoding='utf-8', errors='ignore').read())
parse_dict = merge3dicts(parse_dict_train, parse_dict_dev, parse_dict_test)

parse_data_path = "/home/pengfei/data/2015-2016_conll_shared_task/data/conll16st-en-03-29-16-train/relations.json"
parse_data_dev_path = '/home/pengfei/data/2015-2016_conll_shared_task/data/conll16st-en-03-29-16-dev/relations.json'
parse_data_test_path = '/home/pengfei/data/2015-2016_conll_shared_task/data/conll16st-en-03-29-16-test/relations.json'
parse_data = [json.loads(line) for line in codecs.open(parse_data_path).readlines()]
parse_data_dev = [json.loads(line) for line in codecs.open(parse_data_dev_path).readlines()]
parse_data_test = [json.loads(line) for line in codecs.open(parse_data_test_path).readlines()]
parse_data.extend(parse_data_dev)
parse_data.extend(parse_data_test)

In [8]:
single = 0;double = 0; non = 0
for r in parse_data:
    if len(r['Sense']) == 1:
        single += 1
    if len(r['Sense']) == 2:
        double += 1
    if len(r['Sense']) == 0:
        non += 1
single, double, non

(34667, 1243, 0)

In [9]:
conll_train = '/home/pengfei/data/PDTB-3.0/all/conll/train/pdtb-parses.json'
parse_dict_train = json.loads(codecs.open(conll_train, encoding='utf-8', errors='ignore').read())
conll_dev = '/home/pengfei/data/PDTB-3.0/all/conll/dev/pdtb-parses.json'
parse_dict_dev = json.loads(codecs.open(conll_dev, encoding='utf-8', errors='ignore').read())
conll_test = '/home/pengfei/data/PDTB-3.0/all/conll/test/pdtb-parses.json'
parse_dict_test = json.loads(codecs.open(conll_test, encoding='utf-8', errors='ignore').read())
print("datasets loaded")
parse_dict = merge3dicts(parse_dict_train, parse_dict_dev, parse_dict_test)

parse_data_path = "/home/pengfei/data/PDTB-3.0/all/conll/train/relations.json"
parse_data_dev_path = '/home/pengfei/data/PDTB-3.0/all/conll/dev/relations.json'
parse_data_test_path = '/home/pengfei/data/PDTB-3.0/all/conll/test/relations.json'
parse_data = [json.loads(line) for line in codecs.open(parse_data_path).readlines()]
parse_data_dev = [json.loads(line) for line in codecs.open(parse_data_dev_path).readlines()]
parse_data_test = [json.loads(line) for line in codecs.open(parse_data_test_path).readlines()]
parse_data.extend(parse_data_dev)
parse_data.extend(parse_data_test)

datasets loaded


In [10]:
single = 0;double = 0; non = 0
for r in parse_data:
    if len(r['Sense']) == 1:
        single += 1
    if len(r['Sense']) == 2:
        double += 1
    if len(r['Sense']) == 0:
        non += 1
single, double, non

(45762, 2008, 0)

# conclusion: the double sensed have almost increased